In [ ]:
!pip uninstall opencv-python opencv-python-headless opencv-contrib-python -y

Found existing installation: opencv-python 4.12.0.88
Uninstalling opencv-python-4.12.0.88:
  Successfully uninstalled opencv-python-4.12.0.88
Found existing installation: opencv-python-headless 4.12.0.88
Uninstalling opencv-python-headless-4.12.0.88:
  Successfully uninstalled opencv-python-headless-4.12.0.88
Found existing installation: opencv-contrib-python 4.12.0.88
Uninstalling opencv-contrib-python-4.12.0.88:
  Successfully uninstalled opencv-contrib-python-4.12.0.88


In [ ]:
!pip -q install fvcore opencv-contrib-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 MB 36.1 MB/s eta 0:00:00


In [ ]:
import os, csv, json, cv2, math, time
import numpy as np
import torch
import torch.nn as nn
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional
from collections import defaultdict
from tqdm.auto import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Utilities**

In [ ]:
def _device():
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")

def _clamp(v, lo, hi):
    return max(lo, min(hi, v))

def _expand_bbox(b, margin, W, H):
    """Expand bbox by `margin` while staying in-frame."""
    x1,y1,x2,y2 = map(float, b)
    w, h = x2-x1, y2-y1
    cx, cy = (x1+x2)/2., (y1+y2)/2.
    w2, h2 = w*margin/2., h*margin/2.
    nx1, ny1 = _clamp(int(round(cx - w2)), 0, W-1), _clamp(int(round(cy - h2)), 0, H-1)
    nx2, ny2 = _clamp(int(round(cx + w2)), 0, W-1), _clamp(int(round(cy + h2)), 0, H-1)
    if nx2 <= nx1 or ny2 <= ny1: return 0,0,W-1,H-1
    return nx1, ny1, nx2, ny2

def _center_resize_crop(img, side_size, crop_size):
    """Short-side resize to `side_size`, then center crop `crop_size`."""
    h, w = img.shape[:2]
    if h < w:
        new_h, new_w = side_size, int(round(w*side_size/h))
    else:
        new_w, new_h = side_size, int(round(h*side_size/w))
    rs = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)
    y0 = _clamp((new_h - crop_size)//2, 0, max(0, new_h - crop_size))
    x0 = _clamp((new_w - crop_size)//2, 0, max(0, new_w - crop_size))
    return rs[y0:y0+crop_size, x0:x0+crop_size]

def _linspace_idx(a, b, n):
    """n indices uniformly from [a..b] inclusive, as ints (repeats if n>L)."""
    if n <= 1: return [int(round((a+b)/2))]
    return list(np.round(np.linspace(a, b, n)).astype(int))

def _color_for_id(tid: int):
    # stable-ish distinct color per track id
    return (37*tid % 256, 17*tid % 256, 93*tid % 256)

# **Load tracking CSV**

In [ ]:
def load_tracks_csv(csv_path: str) -> Dict[int, List[Tuple[int, Tuple[int,int,int,int]]]]:
    """
    Returns {track_id: [(frame_idx, (x1,y1,x2,y2)), ...]} sorted by frame.
    """
    print(f"🎬 Loading track data from: {csv_path}")
    tracks = defaultdict(list)
    with open(csv_path, "r") as f:
        reader = csv.DictReader(f)
        # Assuming the CSV isn't too large for an in-memory load
        data = list(reader)
        for r in tqdm(data, desc="   -> Reading CSV lines"):
            t   = int(r["frame"])
            tid = int(r["id"])
            x1,y1,x2,y2 = int(r["x1"]), int(r["y1"]), int(r["x2"]), int(r["y2"])
            tracks[tid].append((t, (x1,y1,x2,y2)))

    print(f"   -> Found {len(tracks)} unique tracks. Sorting by frame...")
    for tid in tracks:
        tracks[tid].sort(key=lambda x: x[0])
    return tracks

def interpolate_track(timeline: List[Tuple[int, Tuple[int,int,int,int]]]):
    """
    Linear interpolation of bboxes across missing frames.
    Returns (t0, t1, dense_boxes: List[(x1,y1,x2,y2)]) covering every frame in [t0..t1].
    """
    ts  = np.array([t for t,_ in timeline], dtype=np.int32)
    bxs = np.array([b for _,b in timeline], dtype=np.float32)  # Nx4
    t0, t1 = int(ts[0]), int(ts[-1])
    T = np.arange(t0, t1+1, dtype=np.int32)
    out = [np.interp(T, ts, bxs[:,k]) for k in range(4)]
    dense = np.stack(out, axis=1).astype(np.int32)  # Lx4
    return t0, t1, [tuple(map(int, b)) for b in dense]

# **Dense TV-L1 flow**

In [ ]:
class TVL1FlowDownscaled:
    """
    Downscale to width<=640 for speed. For each decoded frame, returns flow magnitude map.
    """
    def __init__(self, target_width=640):
        self.tvl1 = cv2.optflow.DualTVL1OpticalFlow_create()
        self.target_width = target_width
        self.prev_gray = None
        self.scale = 1.0
        self.small_shape = None

    def _prep(self, frame_bgr):
        H, W = frame_bgr.shape[:2]
        scale = self.target_width / float(W) if W > self.target_width else 1.0
        small = cv2.resize(frame_bgr, (int(W*scale), int(H*scale)), interpolation=cv2.INTER_AREA) if scale < 1.0 else frame_bgr
        gray = cv2.cvtColor(small, cv2.COLOR_BGR2GRAY)
        return gray, scale, small.shape[:2]

    def mag(self, frame_bgr):
        gray, scale, shp = self._prep(frame_bgr)
        if self.prev_gray is None:
            self.prev_gray = gray
            self.scale = scale
            self.small_shape = shp
            return np.zeros(shp, dtype=np.float32)
        flow = self.tvl1.calc(self.prev_gray, gray, None)
        self.prev_gray = gray
        self.scale = scale
        self.small_shape = shp
        return np.sqrt(flow[...,0]**2 + flow[...,1]**2)

# **Compute actionness**

In [ ]:
@dataclass
class ActionnessCfg:
    """
    Flow-based actionness. Keep simple and robust.
    """
    ma_len: int = 5           # moving-average window on z-scored series
    high_k: float = 1.2       # start when z >= mu + high_k*std
    low_k: float  = 0.4       # stop when z <  mu + low_k*std
    min_dur_s: float = 0.26   # keep segments >= ~0.26s (≈ 8 frames @30fps)
    max_dur_s: float = 1.20   # clamp segments to <= ~1.2s
    bbox_margin: float = 1.25 # enlarge crop a bit so racket/arm stays in

def moving_average(x: np.ndarray, k: int) -> np.ndarray:
    if k <= 1: return x
    w = np.ones(k, dtype=np.float32)/k
    return np.convolve(x, w, mode="same")

def hysteresis_segments(z_sm: np.ndarray, high_k: float, low_k: float) -> List[Tuple[int,int]]:
    mu, sd = float(np.mean(z_sm)), float(np.std(z_sm) + 1e-6)
    high, low = mu + high_k*sd, mu + low_k*sd
    segs, on, s0 = [], False, None
    for i,v in enumerate(z_sm):
        if not on and v >= high:
            on, s0 = True, i
        elif on and v < low:
            on = False
            segs.append((s0, i))
            s0 = None
    if on: segs.append((s0, len(z_sm)-1))
    return segs

def compute_actionness_segments(video_path: str,
                                timeline: List[Tuple[int, Tuple[int,int,int,int]]],
                                fps: float,
                                cfg: ActionnessCfg) -> Tuple[np.ndarray, List[Tuple[int,int]]]:
    """
    Build per-frame actionness inside the player's bbox:
      - TV-L1 flow magnitude (95th percentile) + frame-diff (75th percentile)
      - Per-track z-score, MA smooth, hysteresis
      - Clamp segment durations (min_dur_s .. max_dur_s)

    Returns:
      z_sm  : smoothed z-score array aligned to [t0..t1]
      segsA : list of (t0_abs, t1_abs) frame indices
    """
    # interpolate bboxes densely for this track
    t0, t1, dense_boxes = interpolate_track(timeline)    # inclusive indices
    L = t1 - t0 + 1

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened(): raise RuntimeError(f"Cannot open: {video_path}")
    W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)); H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # We'll decode frames from t0-1 .. t1 to initialize flow
    cap.set(cv2.CAP_PROP_POS_FRAMES, max(0, t0-1))
    flow = TVL1FlowDownscaled(target_width=640)
    mags, diffs = [], []
    prev_small_gray = None

    for t in tqdm(range(max(0,t0-1), t1+1), desc=f"      -> Calculating flow ({L} frames)"):
        ok, frame = cap.read()
        if not ok: break
        mag = flow.mag(frame)               # downscaled mag
        gray_small = cv2.cvtColor(cv2.resize(frame, (mag.shape[1], mag.shape[0]), interpolation=cv2.INTER_AREA),
                                  cv2.COLOR_BGR2GRAY)
        if prev_small_gray is None:
            diff = np.zeros_like(gray_small, dtype=np.float32)
        else:
            diff = cv2.absdiff(gray_small, prev_small_gray).astype(np.float32)
        prev_small_gray = gray_small

        if t >= t0:                         # align to dense series
            mags.append(mag); diffs.append(diff)

    cap.release()
    if len(mags) != L:                      # robust fallback
        mags = [np.zeros(flow.small_shape, np.float32) for _ in range(L)]
        diffs = [np.zeros(flow.small_shape, np.float32) for _ in range(L)]

    # reduce to scalar per frame inside expanded bbox
    z_raw = []
    scale = flow.scale
    for k in range(L):
        b = dense_boxes[k]
        x1,y1,x2,y2 = _expand_bbox(b, cfg.bbox_margin, W, H)
        h_s, w_s = mags[k].shape
        xs1,xs2 = int(x1*scale), int(x2*scale)
        ys1,ys2 = int(y1*scale), int(y2*scale)
        xs1 = _clamp(xs1, 0, w_s-1); xs2 = _clamp(xs2, 0, w_s-1)
        ys1 = _clamp(ys1, 0, h_s-1); ys2 = _clamp(ys2, 0, h_s-1)
        if xs2 <= xs1 or ys2 <= ys1:
            z_raw.append(0.0); continue
        rm = mags[k][ys1:ys2, xs1:xs2]
        rd = diffs[k][ys1:ys2, xs1:xs2]
        # robust percentiles: peak motion > background noise
        m95 = float(np.percentile(rm, 95))
        d75 = float(np.percentile(rd, 75))
        z_raw.append(0.7*m95 + 0.3*d75)

    z_raw = np.asarray(z_raw, dtype=np.float32)
    mu, sd = float(z_raw.mean()), float(z_raw.std() + 1e-6)
    z = (z_raw - mu) / sd
    z_sm = moving_average(z, cfg.ma_len)

    coarse = hysteresis_segments(z_sm, cfg.high_k, cfg.low_k)

    # clamp durations in frames
    min_len = max(1, int(round(cfg.min_dur_s * fps)))
    max_len = max(min_len+1, int(round(cfg.max_dur_s * fps)))

    refined = []
    for a,b in coarse:
        if (b-a+1) < min_len: continue
        # center sub-window around the local peak, limited to max_len
        sub = z_sm[a:b+1]
        peak = int(np.argmax(sub))
        half = max(min((b-a+1)//2, max_len//2), min_len//2)
        c0 = a + max(0, peak - half)
        c1 = a + min(len(sub)-1, peak + half)
        if (c1-c0+1) >= min_len:
            refined.append((c0,c1))

    # merge overlaps and map to absolute indices
    refined = sorted(refined)
    merged = []
    for seg in refined:
        if not merged or seg[0] > merged[-1][1]:
            merged.append(list(seg))
        else:
            merged[-1][1] = max(merged[-1][1], seg[1])
    segs_abs = [(t0 + s0, t0 + s1) for s0,s1 in merged]
    return z_sm, segs_abs

# **Sampling with SlowFast**

In [ ]:
@dataclass
class SFInferCfg:
    slow_t: int = 8            # <- from your cfg
    alpha: int = 4             # <- from your cfg
    side: int = 224            # short-side resize
    crop: int = 224            # center crop size
    mean: Tuple[float,float,float] = (0.45,0.45,0.45)   # Kinetics defaults (pytorchvideo)
    std:  Tuple[float,float,float] = (0.225,0.225,0.225)
    bbox_margin: float = 1.25
    bbox_ema: float = 0.8

def sample_indices(L: int, slow_t: int, alpha: int):
    """
    Produce indices for fast (slow_t*alpha) and slow (slow_t) pathways
    uniformly spanning the segment of length L.
    """
    need_fast = slow_t * alpha
    idx_fast = _linspace_idx(0, max(0, L-1), need_fast)
    idx_slow = idx_fast[::alpha]
    if len(idx_slow) < slow_t:
        idx_slow += [idx_slow[-1]] * (slow_t - len(idx_slow))
    elif len(idx_slow) > slow_t:
        idx_slow = idx_slow[:slow_t]
    return idx_slow, idx_fast

def extract_clip_slowfast(video_path: str,
                          segment: Tuple[int,int],
                          dense_boxes: List[Tuple[int,int,int,int]],
                          sfcfg: SFInferCfg):
    """
    Build (slow, fast) tensors with shapes:
        slow:  (3, slow_t, 224, 224)
        fast:  (3, slow_t*alpha, 224, 224)
    """
    t0, t1 = segment
    L = t1 - t0 + 1

    # Smooth bboxes with EMA to reduce jitter
    smoothed = []
    prev = None
    for b in dense_boxes:
        arr = np.array(b, dtype=np.float32)
        prev = arr if prev is None else sfcfg.bbox_ema*prev + (1-sfcfg.bbox_ema)*arr
        smoothed.append(tuple(prev.astype(int)))

    # Decode frames once
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened(): raise RuntimeError(f"Cannot open {video_path}")
    W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)); H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    cap.set(cv2.CAP_PROP_POS_FRAMES, t0)
    frames = []
    for _ in range(L):
        ok, fr = cap.read()
        if not ok: break
        frames.append(fr)
    cap.release()
    if len(frames) < L and len(frames) > 0:
        frames += [frames[-1]] * (L - len(frames))

    # Crop per frame with margin, then resize/crop to 224
    crops = []
    for k in range(L):
        x1,y1,x2,y2 = _expand_bbox(smoothed[k], sfcfg.bbox_margin, W, H)
        crop = frames[k][y1:y2, x1:x2]
        if crop.size == 0: crop = frames[k]
        crops.append(_center_resize_crop(crop, sfcfg.side, sfcfg.crop))

    # Sample temporal indices
    idx_slow, idx_fast = sample_indices(L, sfcfg.slow_t, sfcfg.alpha)
    slow_frames = [crops[i] for i in idx_slow]
    fast_frames = [crops[i] for i in idx_fast]

    # To tensors (C,T,H,W) normalized
    def _to_tensor(frames_bgr):
        arr = np.stack([cv2.cvtColor(im, cv2.COLOR_BGR2RGB) for im in frames_bgr], axis=0).astype(np.float32)/255.0
        mean = np.array(sfcfg.mean, dtype=np.float32).reshape(1,1,1,3)
        std  = np.array(sfcfg.std,  dtype=np.float32).reshape(1,1,1,3)
        arr = (arr - mean) / std
        arr = np.transpose(arr, (3,0,1,2))  # C,T,H,W
        return torch.from_numpy(arr)

    slow_t = _to_tensor(slow_frames)  # (3, slow_t, 224, 224)
    fast_t = _to_tensor(fast_frames)  # (3, slow_t*alpha, 224, 224)
    return slow_t, fast_t

# **Load SlowFast**

In [ ]:
def load_slowfast_classifier(cfg, ckpt_path: str, device: Optional[torch.device] = None):
    """
    Loads slowfast_r101 (pytorchvideo), replaces head with len(cfg.labels),
    loads checkpoint, sets to eval(). Matches your training code.
    """
    device = device or _device()
    print(f"🧠 Loading SlowFast model from: {ckpt_path}")
    torch.hub._validate_not_a_forked_repo = lambda a,b,c: True
    model = torch.hub.load('facebookresearch/pytorchvideo', 'slowfast_r101', pretrained=True)
    in_dim = model.blocks[-1].proj.in_features
    in_dim = model.blocks[-1].proj.in_features
    model.blocks[-1].proj = nn.Sequential(
        nn.Dropout(p=0.5), # Add a dropout layer
        nn.Linear(in_dim, len(cfg.labels))
    )
    ckpt = torch.load(ckpt_path, map_location=device)
    model.load_state_dict(ckpt["model"], strict=True)
    model.eval().to(device)
    print(f"   -> Model loaded and set to evaluation mode on device: {device}")
    return model

In [ ]:
class SlowFastPredictor:
    """Thin wrapper that accepts lists of (slow,fast) tensors and returns probs."""
    def __init__(self, model, device=None):
        self.model = model.eval()
        self.device = device or _device()

    @torch.no_grad()
    def predict_batch(self, slow_list: List[torch.Tensor], fast_list: List[torch.Tensor]) -> np.ndarray:
        slow = torch.stack(slow_list).to(self.device)  # (B,3,T,H,W)
        fast = torch.stack(fast_list).to(self.device)  # (B,3,T,H,W)
        logits = self.model([slow, fast])              # pytorchvideo expects [slow, fast]
        return torch.softmax(logits, dim=1).cpu().numpy()

In [ ]:
class Config:
    def __init__(self):
        self.root_dir = "/content/drive/MyDrive/FIT3163,3164/SlowFast"
        self.clips_dir = os.path.join(self.root_dir, "05_clips/3in1")
        self.splits_dir = os.path.join(self.root_dir, "06_splits/3in1")
        self.models_dir = os.path.join(self.root_dir, "07_models/3in1_train3")
        self.best_model_path = os.path.join(self.models_dir, "best.pt")

        self.labels = [
            "smash", "jump_smash", "block",
            "drop", "clear", "lift", "drive",
            "straight_net", "cross_net", "serve",
            "push", "tap",
            "average_joe"
        ]

        # Dataset parameters
        self.side = 224             # ori: 224
        self.slow_t = 8             # 8 frames for slow pathway
        self.alpha = 4              # ratio between fast and slow
        self.fast_t = self.slow_t * self.alpha
        self.fast_target = 224      # ori: 224

        # Training parameters
        self.epochs = 30
        self.batch_size = 8
        self.learning_rate = 0.001
        self.weight_decay = 0.001

        self.early_stopping_patience = 5

# Create a configuration object
cfg = Config()

# **Main function**

In [ ]:
def run_full_video_to_events(
    video_path: str,
    csv_path: str,
    cfg,                                 # your Config
    ckpt_path: str,
    action_cfg: Optional[ActionnessCfg] = None,
    sf_cfg: Optional[SFInferCfg] = None
) -> Dict:
    """
    End-to-end:
      1) read StrongSORT CSV
      2) per-track actionness -> segments
      3) per-segment stabilized crops -> SlowFast sampling
      4) batch predict -> events
    Returns dict with 'events' and 'debug'.
    """
    print("=========================================")
    print("🚀 Starting Action Event Inference Pipeline")
    print(f"Video: {os.path.basename(video_path)}")
    print(f"Tracking: {os.path.basename(csv_path)}")
    print("=========================================")

    action_cfg = action_cfg or ActionnessCfg()
    sf_cfg = sf_cfg or SFInferCfg(slow_t=cfg.slow_t, alpha=cfg.alpha, side=cfg.side, crop=cfg.side)

    # 1. Video Meta
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened(): raise RuntimeError(f"Cannot open: {video_path}")
    fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"🎥 Video Meta: FPS={fps:.2f}, Total Frames={total_frames}")
    cap.release()

    # 2. Load Tracking & Model
    tracks = load_tracks_csv(csv_path)
    model = load_slowfast_classifier(cfg, ckpt_path, _device())
    predictor = SlowFastPredictor(model, _device())
    total_tracks = len(tracks)
    print(f"🎯 Inference will run on {total_tracks} tracks.")
    print("-" * 40)

    events = []
    debug = {}

    # 3. Process Tracks
    track_ids = list(tracks.keys())
    for i, tid in enumerate(tqdm(track_ids, desc="Total Track Progress")):
        timeline = tracks[tid]
        print(f"\n   [TRACK {i+1}/{total_tracks}] ID: {tid} ({len(timeline)} frames over {timeline[-1][0]-timeline[0][0]+1} total frames)")

        # 3a. Compute Actionness & Segments
        z_sm, segs = compute_actionness_segments(video_path, timeline, fps, action_cfg)
        debug[int(tid)] = {"t0": int(timeline[0][0]), "z": list(map(float, z_sm)), "segments": [(int(a),int(b)) for a,b in segs]}
        print(f"      -> Found {len(segs)} potential action segments.")
        if not segs:
            continue

        # 3b. Prepare Clips for SlowFast
        t0, t1, dense_boxes = interpolate_track(timeline)
        slow_batch, fast_batch, metas = [], [], []

        # Use a secondary progress bar for segments within the track
        for (a,b) in tqdm(segs, desc="      -> Preparing Segments"):
            if a < t0 or b > t1: continue
            db = dense_boxes[(a - t0):(b - t0 + 1)]
            slow_t, fast_t = extract_clip_slowfast(video_path, (a,b), db, sf_cfg)
            slow_batch.append(slow_t); fast_batch.append(fast_t)
            metas.append((a,b))

        if not slow_batch:
            continue

        # 3c. Batch Predict
        print(f"      -> Predicting {len(slow_batch)} segments in a batch...")
        probs = predictor.predict_batch(slow_batch, fast_batch)  # (B, K)

        # 3d. Collect Events
        for (a,b), p in zip(metas, probs):
            k = int(np.argmax(p))
            events.append({
                "track_id": int(tid),
                "t0": int(a),
                "t1": int(b),
                "label": cfg.labels[k],
                "p": float(p[k])
            })
        print(f"      -> Finished Track {tid}. Found {len(probs)} events.")


    # 4. Final Output
    events.sort(key=lambda e: (e["t0"], e["track_id"]))
    print("\n=========================================")
    print(f"✅ Pipeline Complete. Total Events Found: {len(events)}")
    print("=========================================")
    return {"events": events, "debug": debug}

In [ ]:
INPUT_VIDEO_PATH = "/content/drive/MyDrive/FIT3163,3164/REID/test/reid_test_vid_4.mp4"
CSV_PATH = "/content/annotated_track_4.csv"

device = _device()
sf_model_path = cfg.best_model_path  # from your training

res = run_full_video_to_events(
    video_path=INPUT_VIDEO_PATH,
    csv_path=CSV_PATH,
    cfg=cfg,
    ckpt_path=sf_model_path,
)
with open(os.path.join(cfg.models_dir, "events.json"), "w") as f:
    json.dump(res, f, indent=2)
print(f"Predicted {len(res['events'])} action events")

🚀 Starting Action Event Inference Pipeline
Video: reid_test_vid_4.mp4
Tracking: annotated_track_4.csv


KeyboardInterrupt: 

# **Overlay on video**

In [ ]:
def load_events(events_json_path: str):
    with open(events_json_path, "r") as f:
        data = json.load(f)
    # be flexible (either {"events":[...]} or a plain list)
    events = data["events"] if isinstance(data, dict) and "events" in data else data
    return events

def build_event_map_by_frame(events):
    """
    Returns:
      event_by_frame: {frame_idx: {track_id: best_event_dict}}
      best_event = highest confidence if multiple overlap same frame & track.
    """
    event_by_frame = defaultdict(dict)
    for e in events:
        tid, t0, t1 = int(e["track_id"]), int(e["t0"]), int(e["t1"])
        lab, p = e["label"], float(e.get("p", 1.0))
        for t in range(t0, t1+1):
            cur = event_by_frame[t].get(tid)
            if (cur is None) or (p > float(cur.get("p", 0.0))):
                event_by_frame[t][tid] = {"track_id": tid, "t0": t0, "t1": t1, "label": lab, "p": p}
    return event_by_frame

def load_tracks_csv(csv_path: str):
    """
    Returns:
      boxes_by_frame: {frame_idx: [(tid, (x1,y1,x2,y2)) ...]}
    """
    boxes_by_frame = defaultdict(list)
    with open(csv_path, "r") as f:
        r = csv.DictReader(f)
        for row in r:
            fi = int(row["frame"])
            tid = int(row["id"])
            x1,y1,x2,y2 = int(row["x1"]), int(row["y1"]), int(row["x2"]), int(row["y2"])
            boxes_by_frame[fi].append((tid, (x1,y1,x2,y2)))
    return boxes_by_frame

In [ ]:
def render_full_video_overlay(
    video_path: str,
    tracks_csv: str,
    events_json: str,
    out_path: str,
    show_ids: bool = True,
    label_bg_alpha: float = 0.4
):
    """
    Draws YOLO+StrongSORT boxes and overlays action labels on frames
    that fall inside classified segments for each track.

    - Progress bar drawn above the box shows position inside segment.
    - If track has no active segment on a frame, we draw just the box (and ID).
    """
    boxes_by_frame  = load_tracks_csv(tracks_csv)
    events          = load_events(events_json)
    event_by_frame  = build_event_map_by_frame(events)

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise RuntimeError(f"Cannot open {video_path}")
    fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
    W   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    H   = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    N   = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) or None

    out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (W, H))
    i = 0
    while True:
        ok, frame = cap.read()
        if not ok: break

        # draw tracks for this frame
        items = boxes_by_frame.get(i, [])
        for tid, (x1,y1,x2,y2) in items:
            color = _color_for_id(tid)
            cv2.rectangle(frame, (x1,y1), (x2,y2), color, 2)

            # label overlay if we have an event active now
            ev = event_by_frame.get(i, {}).get(tid)
            if ev is not None:
                label = ev["label"]; p = ev["p"]; t0 = ev["t0"]; t1 = ev["t1"]
                # progress 0..1
                prog = (i - t0) / max(1, (t1 - t0 + 1))
                # translucent box for text
                text = f"{label}  {p:.2f}"
                (tw, th), bl = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
                # place above box if room, else inside
                tx, ty = x1, y1 - 8
                if ty - th - 6 < 0:
                    ty = y1 + th + 12
                # background rect
                bx1, by1 = tx - 2, ty - th - 6
                bx2, by2 = tx + tw + 6, ty + 4
                # alpha blend bg
                bg = frame.copy()
                cv2.rectangle(bg, (bx1,by1), (bx2,by2), color, -1)
                frame = cv2.addWeighted(bg, label_bg_alpha, frame, 1 - label_bg_alpha, 0)
                # text
                cv2.putText(frame, text, (tx, ty), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2, cv2.LINE_AA)

                # progress bar on top edge of bbox
                bar_h = 6
                px1, py1 = x1, max(0, y1 - bar_h - 2)
                px2, py2 = x2, max(0, y1 - 2)
                # bar background (light)
                cv2.rectangle(frame, (px1,py1), (px2,py2), (200,200,200), -1)
                # bar fill with track color
                fill_w = int((px2 - px1) * _clamp(prog, 0.0, 1.0))
                cv2.rectangle(frame, (px1,py1), (px1 + fill_w, py2), color, -1)

            elif show_ids:
                # only ID label
                text = f"ID {tid}"
                (tw, th), bl = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
                tx, ty = x1, y1 - 8
                if ty - th - 6 < 0: ty = y1 + th + 12
                cv2.putText(frame, text, (tx, ty), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        out.write(frame)
        i += 1

    cap.release(); out.release()
    print(f"Overlay saved → {out_path}")

In [ ]:
# EVENTS_JSON = os.path.join(cfg.models_dir, "events.json")
EVENTS_JSON = 'events_1.json'

# render_full_video_overlay(
#     video_path=INPUT_VIDEO_PATH,
#     tracks_csv=CSV_PATH,
#     events_json=EVENTS_JSON,
#     out_path="/content/out_overlay.mp4",
# )

render_full_video_overlay(
    video_path=INPUT_VIDEO_PATH,
    tracks_csv="/content/outputs_webuser_123_manual-test-009_tracks_rectified.csv",
    events_json=EVENTS_JSON,
    out_path="/content/out_overlay.mp4",
)

Overlay saved → /content/out_overlay.mp4
